In [1]:
import h5py
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import os
import numpy as np
import glob

def compute_eer(far, frr):
    cords = zip(far, frr)
    min_dist = 999999
    for item in cords:
        item_far, item_frr = item
        dist = abs(item_far-item_frr)
        if dist < min_dist:
            min_dist = dist
            eer = (item_far + item_frr) / 2
    return eer

# Ped1 ground-truth
testSeqPath = '/usr/not-backed-up/1_DATABASE/Avenue Dataset';
with h5py.File(os.path.join(testSeqPath,'Avenue_FrameLevel_GT.mat')) as f:
    data = [f[element[0]][:] for element in f['gt']]
data = np.array(data)
for seq in range(21):
    gt = data[seq]
    gt_ = gt[4:gt.shape[0],0]
    if seq == 0:
        groundtruth = gt_
    else:
        groundtruth = np.append(groundtruth,gt_)
        
# aug1
decision_path = '/usr/not-backed-up/1_convlstm/avenue_AE6/'

for i in range(21):
    print(i)
    filename = ("test_%d_error.h5" % (i+1))  
    h5f = h5py.File(os.path.join(decision_path,filename), 'r')
    frame_error = h5f['frame_error'][:]
    h5f.close()
    frame_error = frame_error - frame_error.min()
    frame_error = frame_error / frame_error.max()
    frame_error = np.squeeze(frame_error)
    if i == 0:
        error = frame_error
    else:
        error = np.append(error,frame_error)
  
auc = roc_auc_score(groundtruth, error)

fpr, tpr, thresholds = roc_curve(groundtruth, error, pos_label=1)
frr = 1 - tpr
far = fpr
eer = compute_eer(far, frr)
print(eer, auc)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(0.2613082579980327, 0.8143721719986885)
